# Copied from NRCB on Jan 13, 2023

In [ ]:
from photutils import CircularAperture, EPSFBuilder, find_peaks, CircularAnnulus
from photutils.detection import DAOStarFinder, IRAFStarFinder
from photutils.psf import DAOGroup, IntegratedGaussianPRF, extract_stars, IterativelySubtractedPSFPhotometry, BasicPSFPhotometry

In [ ]:
from photutils.background import MMMBackground, MADStdBackgroundRMS
from astropy.modeling.fitting import LevMarLSQFitter

In [ ]:
from astropy import stats
from astropy.table import Table
from astropy.wcs import WCS

In [ ]:
from astropy.io import fits

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.visualization import simple_norm
from astropy import wcs
from astropy import table
from astropy import units as u
import pylab as pl
pl.rcParams['figure.facecolor'] = 'w'
pl.rcParams['image.origin'] = 'lower'

In [ ]:
from astroquery.svo_fps import SvoFps

In [ ]:
import numpy as np

In [ ]:
import reproject

In [ ]:
from astropy import units as u

In [ ]:
basepath = '/orange/adamginsburg/jwst/brick/'

In [ ]:
fh_187 = fits.open(f'{basepath}/F187N/pipeline/jw02221-o001_t001_nircam_clear-f187n-nrca_i2d.fits')
fh_182 = fits.open(f'{basepath}/F182M/pipeline/jw02221-o001_t001_nircam_clear-f182m-nrca_i2d.fits')

In [ ]:
fh_187 = fits.open(f'{basepath}/F187N/pipeline/jw02221-o001_t001_nircam_clear-f187n-nrca_i2d_unsatstar.fits')
fh_182 = fits.open(f'{basepath}/F182M/pipeline/jw02221-o001_t001_nircam_clear-f182m-nrca_i2d_unsatstar.fits')

In [ ]:
ww187 = wcs.WCS(fh_187['SCI'].header)
ww182 = wcs.WCS(fh_182['SCI'].header)

In [ ]:
instrument = fh_187[0].header['INSTRUME']
telescope = fh_187[0].header['TELESCOP']
filt187 = fh_187[0].header['FILTER']
filt182 = fh_182[0].header['FILTER']
instrument, telescope, filt187, filt182

In [ ]:
wavelength_table_187 = SvoFps.get_transmission_data(f'{telescope}/{instrument}.{filt187}')
wavelength_table_182 = SvoFps.get_transmission_data(f'{telescope}/{instrument}.{filt182}')

In [ ]:
filt187, filt182

In [ ]:
waves_182 = wavelength_table_182['Wavelength']
trans_187 = np.interp(waves_182, wavelength_table_187['Wavelength'], wavelength_table_187['Transmission'])
trans_182 = wavelength_table_182['Transmission']

In [ ]:
pl.plot(waves_182, trans_182)
pl.plot(waves_182, trans_187)
pl.xlabel("Wavelength [Angstroms]")

In [ ]:
fractional_bandwidth_187 = ( (trans_182/trans_182.max()) * (trans_187/trans_187.max()) ).sum() / (trans_182/trans_182.max()).sum()

In [ ]:
data_187_proj_182 = reproject.reproject_exact(fh_187['SCI'], fh_182['SCI'].header)

In [ ]:
cont182_sub_paa = fh_182['SCI'].data - data_187_proj_182[0]*fractional_bandwidth_187
fits.PrimaryHDU(data=cont182_sub_paa, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F182_minus_F187_nrca.fits', overwrite=True)

In [ ]:
paa_minus_cont = data_187_proj_182[0] - cont182_sub_paa #* fractional_bandwidth_187
fits.PrimaryHDU(data=paa_minus_cont, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F187_minus_F182cont_nrca.fits', overwrite=True)

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(fh_182['SCI'].data, norm=simple_norm(cont182_sub_paa, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(cont182_sub_paa, norm=simple_norm(cont182_sub_paa, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(fh_187['SCI'].data, norm=simple_norm(cont182_sub_paa, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(paa_minus_cont, norm=simple_norm(paa_minus_cont, min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()

In [ ]:
stars_paa = DAOStarFinder(threshold=60, fwhm=2.302, peakmax=1e4)(fh_187['SCI'].data)
stars_paa['skycoord'] = ww187.pixel_to_world(stars_paa['xcentroid'], stars_paa['ycentroid'])
len(stars_paa)

In [ ]:
stars_182 = DAOStarFinder(threshold=10, fwhm=2.302, peakmax=900)(fh_182['SCI'].data)
stars_182['skycoord'] = ww182.pixel_to_world(stars_182['xcentroid'], stars_182['ycentroid'])
len(stars_182)

In [ ]:
matches, sep, _ = stars_paa['skycoord'].match_to_catalog_sky(stars_182['skycoord'], nthneighbor=1)

for cn in stars_paa.colnames:
    stars_paa.rename_column(cn, f"{cn}_187")
for cn in stars_182.colnames:
    stars_182.rename_column(cn, f"{cn}_182")
                         
stars_paa.add_column(name="sep_182_187", col=sep)
stars_paa.add_column(name="id_182_187", col=matches)
mergetbl = table.hstack([stars_paa, stars_182[matches]], join_type='exact')

In [ ]:
pl.hist(mergetbl['sep_182_187'].to(u.arcsec).value)

In [ ]:
radiff = (mergetbl['skycoord_187'].ra - mergetbl['skycoord_182'].ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_187'].dec - mergetbl['skycoord_182'].dec).to(u.arcsec)
pl.scatter(radiff, decdiff, marker=',', s=1, alpha=0.8)
pl.axis([-0.5,0.5,-0.5,0.5])

# Jan 13, 2023: YIKES:  0.04 x 0.02" major offset!

In [ ]:
pl.figure(figsize=(10,8))
radiff = (mergetbl['skycoord_187'].ra - mergetbl['skycoord_182'].ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_187'].dec - mergetbl['skycoord_182'].dec).to(u.arcsec)
topleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] > 1024)
pl.scatter(radiff[topleft], decdiff[topleft], marker=',', s=1, alpha=0.8)
topright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] > 1024)
pl.scatter(radiff[topright], decdiff[topright], marker=',', s=1, alpha=0.8)
bottomleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] < 1024)
pl.scatter(radiff[bottomleft], decdiff[bottomleft], marker=',', s=1, alpha=0.8)
bottomright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] < 1024)
pl.scatter(radiff[bottomright], decdiff[bottomright], marker=',', s=1, alpha=0.8)
pl.axis([-0.05,0.05,-0.05,0.05])

In [ ]:
mergetbl['xcentroid_187'].max(), mergetbl['ycentroid_187'].max()

In [ ]:
dist_from_center = ((mergetbl['xcentroid_187'] - 2950)**2 + (mergetbl['ycentroid_187'] - 2350)**2)**0.5

In [ ]:
pl.plot(dist_from_center, mergetbl['sep_182_187'].to(u.arcsec).value, ',')
#pl.plot(dist_from_center, dist_from_center/1750 * 0.045 + 0.008)
#pl.plot(dist_from_center, dist_from_center/1750 * 0.035 - 0.004)
pl.ylim(0,0.06)
pl.xlabel("Distance from center of image (pixels)")
pl.ylabel("Offset from F182 (arcseconds)")

# TODO: replace hand-picked numbers here, which are for F405, not F187

In principle, this should be handled by the pipeline tweakwcs steps, but it's not obvious that that has happened

In [ ]:
# selecting 'well-matched' zone (the central blob)
ok = (mergetbl['sep_182_187'] < 0.07*u.arcsec) & (mergetbl['sep_182_187'] < (dist_from_center/1750*0.045 +0.08) *u.arcsec) & (mergetbl['sep_182_187'] > (dist_from_center/1750*0.035 - 0.08) *u.arcsec)

In [ ]:
radiff = (mergetbl['skycoord_187'].ra - mergetbl['skycoord_182'].ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_187'].dec - mergetbl['skycoord_182'].dec).to(u.arcsec)
pl.scatter(radiff[ok], decdiff[ok], marker=',', s=1, alpha=0.8)

In [ ]:
from astropy.wcs.utils import fit_wcs_from_points

Not sure how I chose the SIP degree here.  Investigations of SIP degree are in subsequent cells

In [ ]:
ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok], mergetbl['ycentroid_187'][ok]], mergetbl['skycoord_182'][ok], sip_degree=1)

In [ ]:
ww_182_refit = fit_wcs_from_points([mergetbl['xcentroid_182'][ok], mergetbl['ycentroid_182'][ok]], mergetbl['skycoord_187'][ok], sip_degree=3)

In [ ]:
skycoord_182_refit = ww_182_refit.pixel_to_world(mergetbl['xcentroid_182'], mergetbl['ycentroid_182'])

In [ ]:
pl.figure(figsize=(14,9))
for sip_degree in (0,1,2,3):
    pl.subplot(2,2,sip_degree+1)
    pl.title(f"SIP degree = {sip_degree}")
    
    ww_182_refit = fit_wcs_from_points([mergetbl['xcentroid_182'][ok], mergetbl['ycentroid_182'][ok]], mergetbl['skycoord_187'][ok], sip_degree=sip_degree)
    skycoord_182_refit = ww_182_refit.pixel_to_world(mergetbl['xcentroid_182'], mergetbl['ycentroid_182'])
    radiff = (mergetbl['skycoord_187'].ra - skycoord_182_refit.ra).to(u.arcsec)
    decdiff = (mergetbl['skycoord_187'].dec - skycoord_182_refit.dec).to(u.arcsec)
    sep = (radiff**2 + decdiff**2)**0.5
    ww_182_refit = fit_wcs_from_points([mergetbl['xcentroid_182'][ok & (sep < 0.01*u.arcsec)], mergetbl['ycentroid_182'][ok & (sep < 0.01*u.arcsec)]], mergetbl['skycoord_187'][ok & (sep < 0.01*u.arcsec)], sip_degree=sip_degree)
    skycoord_182_refit = ww_182_refit.pixel_to_world(mergetbl['xcentroid_182'], mergetbl['ycentroid_182'])
    radiff = (mergetbl['skycoord_187'].ra - skycoord_182_refit.ra).to(u.arcsec)
    decdiff = (mergetbl['skycoord_187'].dec - skycoord_182_refit.dec).to(u.arcsec)
    sep = (radiff**2 + decdiff**2)**0.5
    
    topleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] > 1024)
    sc = pl.scatter(radiff[topleft], decdiff[topleft], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[topleft]), np.median(decdiff[topleft]), marker='o', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    topright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] > 1024)
    sc = pl.scatter(radiff[topright], decdiff[topright], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[topright]), np.median(decdiff[topright]), marker='p', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    bottomleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] < 1024)
    sc = pl.scatter(radiff[bottomleft], decdiff[bottomleft], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[bottomleft]), np.median(decdiff[bottomleft]), marker='d', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    bottomright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] < 1024)
    sc = pl.scatter(radiff[bottomright], decdiff[bottomright], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[bottomright]), np.median(decdiff[bottomright]), marker='s', alpha=0.9, c=sc.get_facecolors(), zorder=25, edgecolors='k', s=60)
    pl.axis([-0.01,0.01,-0.01,0.01])

In [ ]:
pl.figure(figsize=(14,9))
for sip_degree in (0,1,2,3):
    pl.subplot(2,2,sip_degree+1)
    pl.title(f"SIP degree = {sip_degree}")
    
    ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok], mergetbl['ycentroid_187'][ok]], mergetbl['skycoord_182'][ok], sip_degree=sip_degree)
    skycoord_187_refit = ww_187_refit.pixel_to_world(mergetbl['xcentroid_187'], mergetbl['ycentroid_187'])
    radiff = (mergetbl['skycoord_182'].ra - skycoord_187_refit.ra).to(u.arcsec)
    decdiff = (mergetbl['skycoord_182'].dec - skycoord_187_refit.dec).to(u.arcsec)
    sep = (radiff**2 + decdiff**2)**0.5
    ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok & (sep < 0.01*u.arcsec)], mergetbl['ycentroid_187'][ok & (sep < 0.01*u.arcsec)]], mergetbl['skycoord_182'][ok & (sep < 0.01*u.arcsec)], sip_degree=sip_degree)
    skycoord_187_refit = ww_187_refit.pixel_to_world(mergetbl['xcentroid_187'], mergetbl['ycentroid_187'])
    radiff = (mergetbl['skycoord_182'].ra - skycoord_187_refit.ra).to(u.arcsec)
    decdiff = (mergetbl['skycoord_182'].dec - skycoord_187_refit.dec).to(u.arcsec)
    sep = (radiff**2 + decdiff**2)**0.5
    
    topleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] > 1024)
    sc = pl.scatter(radiff[topleft], decdiff[topleft], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[topleft]), np.median(decdiff[topleft]), marker='o', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    topright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] > 1024)
    sc = pl.scatter(radiff[topright], decdiff[topright], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[topright]), np.median(decdiff[topright]), marker='p', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    bottomleft = (mergetbl['xcentroid_187'] < 1424) & (mergetbl['ycentroid_187'] < 1024)
    sc = pl.scatter(radiff[bottomleft], decdiff[bottomleft], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[bottomleft]), np.median(decdiff[bottomleft]), marker='d', alpha=0.9, c=sc.get_facecolors(), zorder=15, edgecolors='k', s=60)
    bottomright = (mergetbl['xcentroid_187'] > 1424) & (mergetbl['ycentroid_187'] < 1024)
    sc = pl.scatter(radiff[bottomright], decdiff[bottomright], marker=',', s=1, alpha=0.5)
    pl.scatter(np.median(radiff[bottomright]), np.median(decdiff[bottomright]), marker='s', alpha=0.9, c=sc.get_facecolors(), zorder=25, edgecolors='k', s=60)
    pl.axis([-0.01,0.01,-0.01,0.01])

In [ ]:
sip_degree = 3
ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok], mergetbl['ycentroid_187'][ok]], mergetbl['skycoord_182'][ok], sip_degree=sip_degree)
skycoord_187_refit = ww_187_refit.pixel_to_world(mergetbl['xcentroid_187'], mergetbl['ycentroid_187'])
radiff = (mergetbl['skycoord_182'].ra - skycoord_187_refit.ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_182'].dec - skycoord_187_refit.dec).to(u.arcsec)
sep = (radiff**2 + decdiff**2)**0.5
ww_187_refit = fit_wcs_from_points([mergetbl['xcentroid_187'][ok & (sep < 0.01*u.arcsec)], mergetbl['ycentroid_187'][ok & (sep < 0.01*u.arcsec)]], mergetbl['skycoord_182'][ok & (sep < 0.01*u.arcsec)], sip_degree=sip_degree)
skycoord_187_refit = ww_187_refit.pixel_to_world(mergetbl['xcentroid_187'], mergetbl['ycentroid_187'])
radiff = (mergetbl['skycoord_182'].ra - skycoord_187_refit.ra).to(u.arcsec)
decdiff = (mergetbl['skycoord_182'].dec - skycoord_187_refit.dec).to(u.arcsec)
sep = (radiff**2 + decdiff**2)**0.5


In [ ]:
ww_187_refit.to_header(relax=True).totextfile(f'{basepath}/reduction/headers/f187n_nrca_refitted_to_f182m.hdr', overwrite=True)

In [ ]:
data_187_proj_182_refit, overlap_refit = reproject.reproject_exact((fh_187['SCI'].data, ww_187_refit), fh_182['SCI'].header)

In [ ]:
fits.PrimaryHDU(data=data_187_proj_182_refit, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F187_refitted187wcsto182_nrca.fits', overwrite=True)

In [ ]:
cont182_sub_paa = fh_182['SCI'].data - data_187_proj_182_refit*fractional_bandwidth_187
fits.PrimaryHDU(data=cont182_sub_paa, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F182_minus_F187_refitted187wcsto182_nrca.fits', overwrite=True)

In [ ]:
paa_minus_cont = data_187_proj_182_refit - cont182_sub_paa #* fractional_bandwidth_187
fits.PrimaryHDU(data=paa_minus_cont, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F187_minus_F182cont_refitted187wcsto182_nrca.fits', overwrite=True)

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(cont182_sub_paa, norm=simple_norm(cont182_sub_paa, min_percent=1, max_percent=99, stretch='log'))
pl.colorbar()

In [ ]:
pl.figure(figsize=(12,8))
pl.imshow(paa_minus_cont, norm=simple_norm(paa_minus_cont, min_percent=4, max_percent=99.5, stretch='log'))
pl.colorbar()

In [ ]:
xr = np.linspace(0,15)
pl.scatter(mergetbl['flux_187'][ok], mergetbl['flux_182'][ok], s=1)
pl.plot(xr, xr/fractional_bandwidth_187, color='red')
scalefactor = 0.16
pl.plot(xr, xr/scalefactor, color='orange')
post_destreak_scalefactor = 0.196
pl.plot(xr, xr/post_destreak_scalefactor, color='black')
pl.axis(0,

In [ ]:
latest_scalefactor = np.median(1/(mergetbl['flux_182'][ok]/mergetbl['flux_187'][ok]))
latest_scalefactor

In [ ]:
pl.hist(1/(mergetbl['flux_182'][ok]/mergetbl['flux_187'][ok]), bins=np.linspace(0.05,0.25))
pl.axvline(scalefactor, color='orange')
pl.axvline(post_destreak_scalefactor, color='black')
pl.axvline(fractional_bandwidth_187, color='red')
pl.axvline(latest_scalefactor, color='green', linestyle=':')

In [ ]:
cont182_sub_paa = fh_182['SCI'].data - data_187_proj_182_refit*latest_scalefactor
fits.PrimaryHDU(data=cont182_sub_paa, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F182_minus_F187_refitted187wcsto182_nrca.fits', overwrite=True)

In [ ]:
paa_minus_cont = data_187_proj_182_refit - cont182_sub_paa #* fractional_bandwidth_187
fits.PrimaryHDU(data=paa_minus_cont, header=fh_182['SCI'].header).writeto(f'{basepath}/images/F187_minus_F182cont_refitted187wcsto182_nrca.fits', overwrite=True)